In [1]:
import csv
import pandas
import numpy as np
import pandas as pd
import os.path as osp
import matplotlib.pylab as plt
from pandas import DataFrame
from general_tools.in_out.basics import files_in_subdirs
from general_tools.in_out.basics import pickle_data, unpickle_data

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%matplotlib inline

In [3]:
top_in_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/'
dialogues_dir = osp.join(top_in_dir, 'pilot_chair_data/chatMessage')
dialogue_files = [f for f in files_in_subdirs(dialogues_dir, '.csv')]

In [47]:
'don\'t' in word_dict

False

In [4]:
# game_file = dialogue_files[5]
# in_csv = pd.read_csv(game_file, sep='\t')
# print game_file
# for text, triplet in zip(in_csv['text'], in_csv['intendedName']):
#     print triplet.split('_')[:2]
#     print text + '\n'    

/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/pilot_chair_data/chatMessage/2017-7-19-8h-31m-45s-1994-9aa59dbb-d55b-43bb-8d88-25974d5e073d.csv
['far', '109']
The one that looks the most boxy

['far', '51']
the one that looks like a fence

['close', '95']
the back of the chair is curved upwards

['close', '57']
one of the chairs legs looks awkwardly curved

['far', '2']
you can lay down on it

['close', '86']
there are two folding chairs that look alike. it is the one that has a simpler back.

['far', '91']
looks like a tube

['split', '29']
there are 2 with the fence back. the one where the posts are not at the ends.

['close', '118']
looks like a storage box

['close', '111']
low to the ground with a squiggle piece

['far', '14']
if you sat down and leaned back you would fall

['close', '98']
has some patterns on the back 

['split', '101']
shortest chair

['close', '135']
two chairs look alike. pick the one with the squar back

['far', '114']
little fold up chair

['far', '25'

In [5]:
class Language_Game_Data_Loader(): 
        
    @staticmethod
    def load_dialogues_of_game(game_file):
        special_tokens = {'speaker': ' SpEaKeR: ', 'listener': ' lIsTeNeR: '}    
            
        # Hard-coded incides of interesting quantities.
        triplet_id = 4
        human_role = 5
        sentence_pos = 6
        trial_to_data = {}

        with open(osp.join(top_in_dir, game_file), 'rb') as csvfile:
            reader = csv.reader(csvfile, delimiter='\t')

            for i, row in enumerate(reader):
                if i == 0:                    
                    continue
                    
                tr_id = row[triplet_id].split('_')
                
                if tr_id[0] not in ['far', 'split', 'close'] or not tr_id[1].isdigit():
                    print 'Entry without proper triplet_id was found.'
                    continue

                tr_id = tr_id[0] + '_' + tr_id[1]    
                role = special_tokens[row[human_role]]
                sentence = row[sentence_pos]

                if tr_id in trial_to_data:
                    trial_to_data[tr_id]['dialogue'] += role + sentence
                else:
                    trial_to_data[tr_id] = dict()
                    trial_to_data[tr_id]['dialogue'] = role + sentence
            return trial_to_data
        
    @staticmethod
    def load_selections_of_game(game_file):
        with open(osp.join(top_in_dir, game_file), 'rb') as csvfile:
            reader = csv.reader(csvfile, delimiter='\t')
            for i, row in enumerate(reader):
                if i == 0:                    
                    continue
                print row[15]
    
    @staticmethod
    def _triplet_to_model_map(map_file):
        res = dict()
        with open (map_file, 'r') as fin:
            for line in fin:
                tokens = line.split()
                res[tokens[0]] = (tokens[1], tokens[2], tokens[3])
            return res

    @staticmethod
    def triplet_to_model_map(close_map_file, far_map_file, split_map_file):
        triplet_to_model_map = {
        'close': Language_Game_Data_Loader._triplet_to_model_map(close_map_file),
        'far': Language_Game_Data_Loader._triplet_to_model_map(far_map_file),
        'split': Language_Game_Data_Loader._triplet_to_model_map(split_map_file)}
        return triplet_to_model_map

        
def load_glove_pretrained_model(glove_file):
    print "Loading glove model."
    embedding = dict()
    with open(glove_file, 'r') as f_in:
        for line in f_in:
            s_line = line.split()
            word = s_line[0]
            w_embedding = np.array([float(val) for val in s_line[1:]], dtype=np.float32)
            embedding[word] = w_embedding
    print "Done.", len(embedding), " words loaded!"
    return embedding

In [6]:
close_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/closest_nn/data/model_names_of_triplets.txt'
split_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/one_far/data/model_names_of_triplets.txt'
far_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/both_far/data/model_names_of_triplets.txt'

In [7]:
triplet_to_model_names = Language_Game_Data_Loader.triplet_to_model_map(close_map_file, far_map_file, split_map_file)

In [8]:
language_data = []
for f in dialogue_files:
    language_data.append(Language_Game_Data_Loader.load_dialogues_of_game(f))

Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.
Entry without proper triplet_id was found.


In [9]:
geo_embedding = pandas.read_pickle('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_chair_embeddings.pkl')
word_emb_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Language/glove.6B/glove.6B.100d.txt'
word_dict = load_glove_pretrained_model(word_emb_file)

Loading glove model.
Done. 400000  words loaded!


In [48]:
def embedding_dictionary_to_matrix(in_dict):
    return np.array(in_dict.values())

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

replace_dic = {'!': ' ', ',': ' ', '?': ' ', '.': ' ', ';': ' ', '(': ' ', ')': ' ', '/': ' ',
               '&': 'and', '"': ' ', '-': ' ', '=': ' ', '*': ' ',                
               '\'t': ' not', '\'s': ' of'               
              }

In [11]:
# Deal with "speaker:", "listener:"
# misspelled words
# dentist's chair

In [34]:
# Use tf.nn.embedding_lookup instead and map words to integers
words = set()
words_missed = set()
words_embedded = []
all_data = dict()

for i, game in enumerate(language_data):
    for triplet_id, dialogues in game.iteritems():
        triplet_condition, triplet_num = triplet_id.split('_')
        a, b, c = triplet_to_model_names[triplet_condition][triplet_num]
        triplet_geo_emb = geo_embedding.loc[[a, b, c]].as_matrix().T
        index = np.arange(3)
        np.random.shuffle(index)
        triplet_geo_emb = triplet_geo_emb[:, index]
        label = np.where(index==0)[0][0]
        
        triplet_word_seq = []        
        for w in replace_all(dialogues['dialogue'], replace_dic).split():
            wl = w.lower()
            if wl in word_dict:
                triplet_word_seq.append(word_dict[wl])
                words.add(wl)                
            else: # TODO deal with uknown words
#                 'unk' in word_dict
                words_missed.add(wl)
        
        if len(triplet_word_seq) >= 1:
            if triplet_id not in all_data:
                all_data[triplet_id] = {}
                
            all_data[triplet_id][i] = {}
            all_data[triplet_id][i]['label'] = label
            all_data[triplet_id][i]['word_feats'] =  np.array(triplet_word_seq)
            all_data[triplet_id][i]['geo_feats'] = triplet_geo_emb
            all_data[triplet_id][i]['condition'] = triplet_condition

In [22]:
unique_targets = set()
for key in all_data:
    unique_targets.add(key.split('_')[1])

unique_targets = np.array(list(unique_targets), dtype=object)

In [23]:
conditions = ['far', 'split', 'close']
samples_per_target = []
for target in unique_targets:
    t_samples = 0
    for cond in conditions:        
        key = cond + '_' + target
        if key in all_data:
            t_samples += len(all_data[key])
    samples_per_target.append(t_samples)

In [35]:
# exclude_for_test_data = np.argsort(samples_per_target)[:10]   % those with fewest samples

exclude_for_test_data = np.random.choice(len(unique_targets), 10, replace=False)
exclude_for_test_data = unique_targets[exclude_for_test_data]
print exclude_for_test_data


test_data = {}
train_data = {}

for triplet_id in all_data:
    target = triplet_id.split('_')[1]
    
    if target in exclude_for_test_data:
        test_data[triplet_id] = all_data[triplet_id]
    else:
        train_data[triplet_id] = all_data[triplet_id]

['138' '135' '130' '25' '11' '5' '95' '118' '22' '113']


In [50]:
def sequences_to_numpy_arrays(in_seq_data, max_seq_len=30, word_feat_dim=100, geo_feat_dim=128):
    n_triplets = 0
    for triplet_id in in_seq_data:
        n_triplets += len(in_seq_data[triplet_id])
    
    word_data = np.zeros(shape=(n_triplets, max_seq_len, word_feat_dim))
    all_seq_len = []

    i = 0
    for triplet_id in in_seq_data:
        for key in in_seq_data[triplet_id]: # iterate over different users/players
            datum = in_seq_data[triplet_id][key]['word_feats']
            n_steps = datum.shape[0]
            all_seq_len.append(n_steps)        
            pad_many = max_seq_len-n_steps
            if pad_many >= 0:
                word_data[i] = np.pad(datum, ((0, pad_many), (0,0)) , 'constant')
            elif pad_many < 0:
                word_data[i] = datum[:max_seq_len,:]
            i += 1
    
    geo_data = np.zeros(shape=(n_triplets, 3*geo_feat_dim))
    label_data = np.zeros((n_triplets, 3), dtype=np.int32)
    condition_data = np.empty(shape=(n_triplets,), dtype=object)
    
    i = 0
    for triplet_id in in_seq_data:
        for key in in_seq_data[triplet_id]:
            datum = in_seq_data[triplet_id][key]['geo_feats']       
            geo_data[i] = np.hstack(datum)
            
            label = in_seq_data[triplet_id][key]['label']
            label_data[i, label] = 1
            
            condition_data[i] = in_seq_data[triplet_id][key]['condition'] # TEMP
            i += 1
    return word_data, geo_data, label_data, condition_data

In [51]:
class GeoWordsDataSet(object):

    def __init__(self, words, geometries, labels, conditions, use_copies=True, init_shuffle=False):
        self.num_examples = words.shape[0]
        assert(self.num_examples == labels.shape[0])
        assert(self.num_examples == geometries.shape[0])
        
        assert( use_copies )
        assert(init_shuffle==False)
        
        self.words = words.copy()        
        self.geometries = geometries.copy()
        self.labels = labels.copy()
        self.conditions = conditions.copy()

        self.epochs_completed = 0
        self._index_in_epoch = 0
        if init_shuffle:
            self.shuffle_data()
            
    def shuffle_data(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        perm = np.arange(self.num_examples)
        np.random.shuffle(perm)
        self.words = self.words[perm]
        self.geometries = self.geometries[perm]
        self.labels = self.labels[perm]
        return self

    def next_batch(self, batch_size, seed=None):
        '''Return the next batch_size examples from this data set.
        '''
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self.num_examples:
            self.epochs_completed += 1  # Finished epoch.
            self.shuffle_data(seed)
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
        end = self._index_in_epoch

        return self.words[start:end], self.geometries[start:end], self.labels[start:end], self.conditions[start:end]

In [52]:
word_data, geo_data, label_data, cond_data = sequences_to_numpy_arrays(train_data)
gw_data = GeoWordsDataSet(words=word_data, geometries=geo_data, labels=label_data, conditions=cond_data)
pickle_data('first_pilot_training_data.pkl', gw_data)

word_data, geo_data, label_data, cond_data = sequences_to_numpy_arrays(test_data)
gw_data = GeoWordsDataSet(words=word_data, geometries=geo_data, labels=label_data, conditions=cond_data)
pickle_data('first_pilot_test_data.pkl', gw_data)

array([None, None, None, None, None, None, None, None, None, None], dtype=object)

In [101]:
# map numbers to language
# typos


# sqare -> square 
# sqiggle -> squiggly
# shorest -> shortest
# pilow -> pillow